In [1]:
import pandas as pd
import os

## Data

In [2]:
file_path = "./AAPL_daily2.csv"
data = pd.read_csv(file_path, index_col=0)

In [3]:
x = data.iloc[:,0:6]
y = data.iloc[:,7]

features = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']
print(features)

x.columns = features

['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']


In [4]:
x_train,x_test = x.iloc[:5000,:], x.iloc[5000:,:]
y_train,y_test = y[:5000],y[5000:]

## XGB model

In [5]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(x_train, y_train)

/home/aswin/anaconda3/envs/xg_lleaves/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/aswin/anaconda3/envs/xg_lleaves/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/aswin/anaconda3/envs/xg_lleaves/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [6]:
# Saving the model
model_xgb._Booster.dump_model("xgb_aapl.txt")
model_xgb.save_model("xgb_aapl.json")

In [7]:
xgb_prediction = model_xgb.predict(x_test)

In [8]:
xgb_prediction

array([ 0.03996581, -0.28144288,  0.49001595, ...,  0.7893067 ,
        0.71400946,  0.75283796], dtype=float32)

In [9]:
%%timeit -n 10 -r 3

xgb_prediction = model_xgb.predict(x)

9.27 ms ± 1.77 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


## lleaves model

In [10]:
from lleaves.compiler.ast.scanner import scan_model_file
import lleaves
# current model only works with hardcoded features(aapl stock data is used here in this case)

In [11]:
scanned_md = scan_model_file("xgb_aapl.txt", features)

In [12]:
model = lleaves.Model("xgb_aapl.txt")

In [13]:
model.compile()

In [14]:
our_predictions = model.predict(x_test)

In [15]:
our_predictions + 0.5

array([ 0.03996586, -0.28144285,  0.49001607, ...,  0.7893066 ,
        0.71400942,  0.75283789])

In [16]:
%%timeit -n 10 -r 3

pur_predictions_prediction = model.predict(x)

1.56 ms ± 97.8 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
